In [1]:
#cell-0 imports,some parameters set,paths
import numpy as np
import pandas as pd
import os
import glob
import csv
from pathlib import Path
import cv2 #for frame resizing and optical flow
from skimage.feature import hog #for accesing HOG A
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import    IsolationForest

from tqdm import tqdm #for seeing the progress as a percentage 

#frame resolution
FRAME_WIDTH=256
FRAME_HEIGHT=256
FRAME_SIZE=(FRAME_WIDTH,FRAME_HEIGHT)
#PARAMETERS FOR OPTICAL FLOW 
Flow_parameters=dict(orientations=9,pixels_per_cell=(8*8),cells_per_block=(2,2),block_norm='L2-Hys',feature_vector=True)

#TEMPORAL FRAME OFFSETS FOR MOTION MODELLING
Delta_short=1 #short term motion (t-1)
Delta_long=5#long term motion(t-5)
#PATHS
TRAIN_VIDEOS_DIR = "/kaggle/input/pixel-play-26/Avenue_Corrupted-20251221T112159Z-3-001/Avenue_Corrupted/Dataset/training_videos"
TEST_VIDEOS_DIR  =  "/kaggle/input/pixel-play-26/Avenue_Corrupted-20251221T112159Z-3-001/Avenue_Corrupted/Dataset/testing_videos"
SUBMISSION_PATH="submission.csv"


In [ ]:
# cell-1A: load videos, fix flipped frames, resize, grayscale

def load_and_preprocess_videos(videos_dir):

    processed_videos = {}
    flipped_frames = []

    video_folders = sorted([d for d in os.listdir(videos_dir) if d.isdigit()], key=lambda x: int(x))
    
    for video_id in tqdm(video_folders):

        video_path = os.path.join(videos_dir, video_id)
        frame_files = sorted(
            glob.glob(os.path.join(video_path, "*.jpg")),
            key=lambda x: int(os.path.splitext(os.path.basename(x))[0])
        )

        frames_gray = []
        frame_ids = []

        for frame_path in frame_files:

            name = os.path.basename(frame_path)
            num = int(os.path.splitext(name)[0])
            frame_id = video_id + "_" + str(num)

            frame = cv2.imread(frame_path)
            if frame is None:
                continue

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)#RGB TO GRAYSCALE
            h, w = gray.shape

            # computING  edge energy using sobel
            sobel_y = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3)

            top_energy = np.mean(np.abs(sobel_y[:h//3, :]))
            bottom_energy = np.mean(np.abs(sobel_y[2*h//3:, :]))

            # if top is greater than bottom, frame  flipped
            if top_energy > bottom_energy:
                frame = cv2.flip(frame, -1)
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                flipped_frames.append(frame_id)

            # resize to 256*256 (128*128 didnt worked well) after fixing orientation
            gray = cv2.resize(gray, FRAME_SIZE, interpolation=cv2.INTER_AREA)

            frames_gray.append(gray)
            frame_ids.append(frame_id)

        processed_videos[video_id] = {
            "frames": frames_gray,
            "ids": frame_ids
        }

    print("Total flipped frames corrected:", len(flipped_frames))
    

    return processed_videos


# apply preprocessing

train_data = load_and_preprocess_videos(TRAIN_VIDEOS_DIR)
test_data  = load_and_preprocess_videos(TEST_VIDEOS_DIR)


print('preprocessing done')


In [ ]:
# cell-2:  HOG feature extraction

def extracting_hog_features(video_data):

    hog_features = []
    frame_ids = []

    for video_id in tqdm(video_data.keys()):

        frames = video_data[video_id]["frames"]

        ids    = video_data[video_id]["ids"]

        for gray, fid in zip(frames, ids):

            # getting HOG feature  (appearance features)
            hog_feat = hog(gray, **HOG_PARAMS)

            hog_features.append(hog_feat)

            frame_ids.append(fid)

    hog_features = np.array(hog_features)
    return hog_features, frame_ids
# applying on train and test

hog_train_raw, hog_train_ids = extracting_hog_features(train_data)
hog_test_raw,  hog_test_ids  = extracting_hog_features(test_data)
print("hog feature matrix made now pca to be applied")


In [ ]:
# cell-3:applying  PCA on  HOG features 
hog_pca = PCA( n_components=128,whiten=True,random_state=42) #per frame now only 128 most imp features remain and whiten true so that all 128 compents gets var=1 and random_state=42 so that rerunning does not change results

# fitting  only on training data and not test to prevent data leakage and then transforming train data 
hog_train_pca = hog_pca.fit_transform(hog_train_raw)

#test data getting same redn technique based on train 
hog_test_pca = hog_pca.transform(hog_test_raw)
print('dimensionality redn done ')









In [ ]:
# cell-4: extracting optical flow based 21 motion features

def extracting_optical_flow_features(video_data):

    all_flow_features = []
    all_frame_ids = []

    

    for video_id in tqdm(video_data.keys()):

        frames = video_data[video_id]["frames"]
        ids    = video_data[video_id]["ids"]
        prev_gray = None
        prev_mag  = None

        for gray, fid in zip(frames, ids):

            # first frame has no previous reference so filling it 21D as zero
            if prev_gray is None:
                all_flow_features.append(np.zeros(21, dtype=np.float32))
                all_frame_ids.append(fid)
                prev_gray = gray
                prev_mag = np.zeros_like(gray, dtype=np.float32)
                continue


            # dense optical flow 
            flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, **FLOW_PARAMS)

            vx = flow[..., 0]
            vy = flow[..., 1]
            #convering vx and vy to to polar speed(mag) and angle(ang)
            mag, ang = cv2.cartToPolar(vx, vy)

            # calculating 8 mag related  features for a frame
            mag_mean = mag.mean()
            mag_std  = mag.std()
            mag_p50  = np.percentile(mag, 50)
            mag_p75  = np.percentile(mag, 75)
            mag_p90  = np.percentile(mag, 90)
            mag_p95  = np.percentile(mag, 95)
            mag_max  = mag.max()
            mag_sparse = np.mean(mag > mag_p90)

            mag_features = [mag_mean, mag_std, mag_p50, mag_p75, mag_p90, mag_p95, mag_max, mag_sparse]
                           

            # cal 8 angle related features and 1 entropy for detecting randomness
            dir_hist, _ = np.histogram(ang, bins=8, range=(0, 2*np.pi), density=True)
            dir_entropy = -np.sum(dir_hist * np.log(dir_hist + 1e-6))

            # cal coherence feature 
            mean_vx = vx.mean()
            mean_vy = vy.mean()
            coherence = np.sqrt(mean_vx**2 + mean_vy**2) / (mag_mean + 1e-8)#1e-8 bcz when i ran first time i got some error bcz mag.mean is zero for all video's first frame

            # 3 accelration related features making
            acc = mag - prev_mag#change in speed no time term bcz i treat time gap of 1 frama as 1s
            acc_mean = acc.mean()
            acc_std  = acc.std()
            acc_p90  = np.percentile(acc, 90)

            acc_features = [acc_mean, acc_std, acc_p90]

            # all 21 d feature matrix
            flow_feat = np.concatenate([mag_features, dir_hist.tolist(),   [dir_entropy],[coherence],acc_features ])

            all_flow_features.append(flow_feat)
            all_frame_ids.append(fid)
            #updating frames and speed 
            prev_gray = gray
            prev_mag  = mag

    flow_features = np.array(all_flow_features, dtype=np.float32)

    return flow_features, all_frame_ids


# applying it on  on train and test sets 

flow_train, flow_train_ids = extracting_optical_flow_features(train_data)
flow_test,  flow_test_ids  = extracting_optical_flow_features(test_data)
print("completed all 21 optical featrires")


In [ ]:
# cell-5: temporal smoothing and 21to 63 d conversion of optical flow features

def temporal_smooth_and_deltas(flow_features, video_data):

    enhanced_features = []
    enhanced_ids = []

    cursor = 0   # pointer over flat flow feature list

    for video_id in tqdm(video_data.keys()):

        frames = video_data[video_id]["frames"]
        ids    = video_data[video_id]["ids"]

        num_frames = len(frames)

        # extracting flow features for this video only so that features of one video does not mix with other
        video_flow = flow_features[cursor: cursor + num_frames]
        cursor += num_frames

        #applying smoothing 3 window smoothing to avoid uncertain spikes in normal motion bcz some images were kind of blurredor high vingnette so this will help to avoid
        smoothed = []

        for t in range(num_frames):
            if t == 0 or t == num_frames - 1:
                smoothed.append(video_flow[t])
            else:
                smoothed.append((video_flow[t-1] + video_flow[t] + video_flow[t+1]) / 3)

        smoothed = np.array(smoothed)

        # using delta logic 
        for t in range(num_frames):

            base = smoothed[t]

            # delta (t-1)
            if t - DELTA_SHORT >= 0:
                delta_short = base - smoothed[t - DELTA_SHORT]
            else:
                delta_short = np.zeros_like(base)

            # delta (t-5)
            if t - DELTA_LONG >= 0:
                delta_long = base - smoothed[t - DELTA_LONG]
            else:
                delta_long = np.zeros_like(base)

            # final 63-D feature
            enhanced = np.concatenate([base, delta_short, delta_long])

            enhanced_features.append(enhanced)
            enhanced_ids.append(ids[t])

    enhanced_flow_features = np.array(enhanced_features, dtype=np.float32)

    return enhanced_flow_features, enhanced_ids


# applying this  on train and test

flow_train_enhanced, flow_train_ids = temporal_smooth_and_deltas(flow_train, train_data)
flow_test_enhanced,  flow_test_ids  = temporal_smooth_and_deltas(flow_test, test_data)


print("Temporal smoothing + deltas completed move to next cell")


In [ ]:
# cell-6: mixing 128 hog and 63 flow features 
X_train = np.concatenate([hog_train_pca, flow_train_enhanced], axis=1)
X_test  = np.concatenate([hog_test_pca,  flow_test_enhanced],  axis=1)


# normalizing bcz these 191 features can have very diff scales
scaler = StandardScaler()

# fit only on training data
X_train_scaled = scaler.fit_transform(X_train)

# apply same scaling to test
X_test_scaled = scaler.transform(X_test)
# storing  final ids
final_train_ids = hog_train_ids
final_test_ids  = hog_test_ids

print('feature matrix ready for isoln forest')

In [ ]:


# cell-7: isolation forest training + per-video anomaly normalization

iso_forest = IsolationForest(
    n_estimators=400,
    max_samples="auto",
    contamination=0.10,
    max_features=1.0,
    bootstrap=False,
    n_jobs=-1,
    random_state=42
)

# train only on training (mostly normal) data
iso_forest.fit(X_train_scaled)

print(" training done")


# raw anomaly scores
# decision_function: higher = more normal
# multiply by -1 so higher = more anomalous

train_scores_raw = -iso_forest.decision_function(X_train_scaled)
test_scores_raw  = -iso_forest.decision_function(X_test_scaled)


#  per-video normalization 

def per_video_normalize(scores, video_data, frame_ids):

    norm_scores = np.zeros_like(scores, dtype=np.float32)

    cursor = 0

    for video_id in video_data.keys():

        num_frames = len(video_data[video_id]["frames"])
        video_scores = scores[cursor : cursor + num_frames]

        vmin = video_scores.min()
        vmax = video_scores.max()

        video_scores_norm = (video_scores - vmin) / (vmax - vmin + 1e-8)

        norm_scores[cursor : cursor + num_frames] = video_scores_norm
        cursor += num_frames

    return norm_scores


# apply per-video normalization
final_train_scores = per_video_normalize(train_scores_raw, train_data, final_train_ids)
final_test_scores  = per_video_normalize(test_scores_raw,  test_data,  final_test_ids)




In [ ]:
# cell-8: submission csv generation
# building rows

submission_rows = list(zip(final_test_ids, final_test_scores))
assert len(submission_rows) == TOTAL_EXPECTED_ROWS
#writing csv 
with open(SUBMISSION_PATH, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Id", "Predicted"])

    for Id, score in submission_rows:
        writer.writerow([Id, float(score)])

print("Submission file saved to:", SUBMISSION_PATH)


In [ ]:
#boosting the natural csv
import pandas as pd
import numpy as np

INPUT_CSV  = "submission.csv"
OUTPUT_CSV = "boosted_submission.csv"

TOP_PCT = 0.01     
BOOST   = 0.15     

df = pd.read_csv(INPUT_CSV)
scores = df["Predicted"].values.copy()

N = len(scores)
k = max(1, int(TOP_PCT * N))

top_idx = np.argsort(scores)[-k:]
scores[top_idx] = scores[top_idx] + BOOST


scores = (scores - scores.min()) / (scores.max() - scores.min() + 1e-8)


out = df.copy()
out["Predicted"] = scores
out.to_csv(OUTPUT_CSV, index=False)

